# Analise Analitica descritiva dos Dados

iremos separar os dados nos seguintes tipos:

| Tipo         | Exemplos no Pandas  | Usos comuns                          |
| ------------ | ------------------- | ------------------------------------ |
| `object`     | texto, rótulos      | categóricos nominais, strings        |
| `category`   | valores categóricos | categóricos nominais ou ordinais     |
| `int64`      | inteiros            | dados discretos, contagens           |
| `datetime64` | datas               | séries temporais, análises sazonais  |

In [4]:
import pandas as pd
import numpy as np
import sqlite3
import statistics as st
import scipy

In [5]:
def carrega_tabela(database, tabela):
    '''
    Carrega uma tabela de um banco de dados 
    SQLite em um DataFrame do Pandas.
    Parâmetros:
    database: str - Caminho para o arquivo do banco de dados SQLite.
    tabela: str - Nome da tabela a ser carregada.
    '''
    with sqlite3.connect(database) as conn:
        query = f"SELECT * FROM {tabela}"
        df = pd.read_sql_query(query, conn)
    return df

In [6]:
def lista_tabelas(db_filename):
    '''
    Lista todas as tabelas em um banco de dados SQLite.
    Parâmetros:
    db_filename: str - Caminho para o arquivo do banco de dados SQLite.
    Retorna:
    DataFrame com os nomes das tabelas e suas contagens de linhas.
    '''
    # Connect to the SQLite database
    # conn = sqlite3.connect(db_filename)
    with sqlite3.connect(db_filename) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        table_row_counts = []
        for table in tables:
            table_name = table[0]
            query = f"SELECT COUNT(*) FROM {table_name};"
            cursor.execute(query)
            row_count = cursor.fetchone()[0]
            table_row_counts.append({"Table": table_name, "Row_Count": row_count})

        return pd.DataFrame(table_row_counts)

In [5]:
lista_tabelas('bilheteria.db')

,Table,Row_Count
0,distribuidora,72
1,filme,515
2,grupo_exibidor,64
3,exibidor,180
4,complexo,683
5,sala,3231
6,sessao,1748363


In [36]:
sessao = carrega_tabela("bilheteria.db","sessao")
sala = carrega_tabela("bilheteria.db","sala")
complexo = carrega_tabela("bilheteria.db","complexo")
#exibidor = carrega_tabela("bilheteria.db","exibidor")
#grupo_exibidor = carrega_tabela("bilheteria.db","grupo_exibidor")
#filme = carrega_tabela("bilheteria.db","filme")
#distribuidora = carrega_tabela("bilheteria.db","distribuidora")

# Sessao

In [39]:
#sessao.groupby("publico")["publico"].value_counts().plot()
sessao[sessao.sala_id == 5001791].groupby("filme_id")["filme_id"].value_counts().sort_values()

filme_id
150     1
250     1
328     1
289     1
472     2
       ..
293    28
288    29
274    36
125    41
136    48
Name: count, Length: 92, dtype: int64

In [10]:
sessao["data_exibicao"] = pd.to_datetime(sessao["data_exibicao"], format="%d/%m/%Y")
sessao["filme_id"] = sessao["filme_id"].astype("category")
sessao["sala_id"] = sessao["sala_id"].astype("category")

Aqui devemos entender os tipos de dados que temos

In [21]:
print(sessao.shape)
print(sessao.info())
print(sessao.head())

print(sessao.filme_id.value_counts().head(1))
print(sessao.sala_id.value_counts().head(1))
print(sessao.data_exibicao.value_counts().head(1))

print(sessao[["publico", "data_exibicao"]].describe())
print("Moda da coluna: ",sessao.publico.value_counts().head(5))
print("Moda da coluna: ",sessao.data_exibicao.value_counts().head(5))
print("Moda da coluna: ",sessao.filme_id.value_counts().head(5))
print("Moda da coluna: ",sessao.sala_id.value_counts().head(5))


(1748363, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1748363 entries, 0 to 1748362
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   id             int64 
 1   filme_id       int64 
 2   sala_id        int64 
 3   publico        int64 
 4   data_exibicao  object
dtypes: int64(4), object(1)
memory usage: 66.7+ MB
None
   id  filme_id  sala_id  publico data_exibicao
0   1       253  5004897        8    01/01/2023
1   2       253  5001565       15    01/01/2023
2   3       253  5001566       14    01/01/2023
3   4       253  5003464       23    01/01/2023
4   5       253  5001137       37    01/01/2023
filme_id
256    70676
Name: count, dtype: int64
sala_id
5001791    1176
Name: count, dtype: int64
data_exibicao
08/07/2023    6552
Name: count, dtype: int64
            publico
count  1.748363e+06
mean   6.300355e+01
std    9.562188e+01
min    1.000000e+00
25%    1.100000e+01
50%    3.000000e+01
75%    7.500000e+01
max    3.501000e+03
Moda 

# Sala

In [9]:
sala

,id,nome,from_complexo
0,5000056,KINOPLEX IGUAÇU TOP SALA 01 - KINOEVOLUTION,2387
1,5000057,KINOPLEX IGUAÇU TOP SALA 02,2387
2,5000058,KINOPLEX IGUAÇU TOP SALA 03,2387
3,5000061,SALA KINOPLEX MADUREIRA 01,2389
4,5000062,SALA KINOPLEX MADUREIRA 02,2389
...,...,...,...
3226,5007071,MULTICINE SÃO JOSÉ 2,55466
3227,5007072,MULTICINE SÃO JOSÉ 3,55466
3228,5007073,MULTICINE SÃO JOSÉ 4,55466
3229,5007075,CINE RITZ MONLEVADE 1,55563


In [15]:
sala.from_complexo = sala.from_complexo.astype("category")

In [43]:
print("Linhas e Colunas: ",sala.shape)
print("Primeiras 5 linhas: ",sala.head())
print("indices das colunas",sala.columns)
print("tipos de dados apos editado: ", sala.info())

print("Moda da coluna: ",sala.from_complexo.value_counts().head(1))

Linhas e Colunas:  (3231, 3)
Primeiras 5 linhas:          id                                         nome from_complexo
0  5000056  KINOPLEX IGUAÇU TOP SALA 01 - KINOEVOLUTION          2387
1  5000057                  KINOPLEX IGUAÇU TOP SALA 02          2387
2  5000058                  KINOPLEX IGUAÇU TOP SALA 03          2387
3  5000061                   SALA KINOPLEX MADUREIRA 01          2389
4  5000062                   SALA KINOPLEX MADUREIRA 02          2389
indices das colunas Index(['id', 'nome', 'from_complexo'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3231 entries, 0 to 3230
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   id             3231 non-null   int64   
 1   nome           3231 non-null   object  
 2   from_complexo  3231 non-null   category
dtypes: category(1), int64(1), object(1)
memory usage: 78.4+ KB
tipos de dados apos editado:  None
Moda da coluna:  from_com

# Complexo

In [7]:
complexo[complexo.id == 2620]

,id,municipio,UF,from_exibidor
56,2620,RIO DE JANEIRO,RJ,749


In [25]:
complexo.municipio = complexo.municipio.astype("category")
complexo.UF = complexo.UF.astype("category")
complexo.from_exibidor = complexo.from_exibidor.astype("category")

In [27]:
print("Linhas e Colunas: ",complexo.shape)
print("Primeiras 5 linhas:\n",complexo.head())
print("indices das colunas",complexo.columns)
print("tipos de dados apos editado: ", complexo.info())

print("Moda da coluna: ",complexo.municipio.value_counts().head(1))
print("Moda da coluna: ",complexo.UF.value_counts().head(1))
print("Moda da coluna: ",complexo.from_exibidor.value_counts().head(1))

Linhas e Colunas:  (683, 4)
Primeiras 5 linhas:
     id       municipio  UF from_exibidor
0  438  BELO HORIZONTE  MG           437
1  895       SÃO PAULO  SP          1843
2  896     SANTO ANDRÉ  SP          1843
3  897       SÃO PAULO  SP          1843
4  898       SÃO PAULO  SP          1843
indices das colunas Index(['id', 'municipio', 'UF', 'from_exibidor'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683 entries, 0 to 682
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   id             683 non-null    int64   
 1   municipio      683 non-null    category
 2   UF             683 non-null    category
 3   from_exibidor  683 non-null    category
dtypes: category(3), int64(1)
memory usage: 26.3 KB
tipos de dados apos editado:  None
Moda da coluna:  municipio
SÃO PAULO    65
Name: count, dtype: int64
Moda da coluna:  UF
SP    198
Name: count, dtype: int64
Moda da coluna:  from_exibidor
18

# exibidor

In [11]:
exibidor

,id,from_grupo
0,260,6000049
1,437,6000013
2,592,6000030
3,749,6000058
4,1660,6000046
...,...,...
175,52193,6000161
176,52290,6000104
177,53307,6000040
178,53478,6000040


In [29]:
exibidor.from_grupo = exibidor.from_grupo.astype("category")

In [31]:
print("Linhas e Colunas: ",exibidor.shape)
print("Primeiras 5 linhas:\n",exibidor.head())
print("indices das colunas",exibidor.columns)
print("tipos de dados apos editado: ", exibidor.info())

print("Moda da coluna: ",exibidor.from_grupo.value_counts().head(1))

Linhas e Colunas:  (180, 2)
Primeiras 5 linhas:
      id from_grupo
0   260    6000049
1   437    6000013
2   592    6000030
3   749    6000058
4  1660    6000046
indices das colunas Index(['id', 'from_grupo'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   id          180 non-null    int64   
 1   from_grupo  180 non-null    category
dtypes: category(1), int64(1)
memory usage: 4.3 KB
tipos de dados apos editado:  None
Moda da coluna:  from_grupo
6000000    16
Name: count, dtype: int64


# Grupo Exibidor

In [12]:
grupo_exibidor

,id
0,5002765
1,6000000
2,6000002
3,6000003
4,6000006
...,...
59,6000122
60,6000140
61,6000141
62,6000161


In [33]:
print("Linhas e Colunas: ",grupo_exibidor.shape)
print("Primeiras 5 linhas:\n",grupo_exibidor.head())
print("indices das colunas",grupo_exibidor.columns)
print("tipos de dados apos editado: ", grupo_exibidor.info())

Linhas e Colunas:  (64, 1)
Primeiras 5 linhas:
         id
0  5002765
1  6000000
2  6000002
3  6000003
4  6000006
indices das colunas Index(['id'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      64 non-null     int64
dtypes: int64(1)
memory usage: 644.0 bytes
tipos de dados apos editado:  None


# Filme

In [13]:
filme

,id,titulo_original,titulo_br,cpb_roe,pais_origem,from_distribuidora
0,1,DURVAL DISCOS,None,B0200001000000,BRASIL,24
1,2,NOSSO LAR,None,B1001259400000,BRASIL,3
2,3,REMOÇÃO,None,B1301918400000,BRASIL,51
3,4,HOJE EU QUERO VOLTAR SOZINHO,None,B1402094700000,BRASIL,24
4,5,O HOMEM DE LAGOA SANTA,None,B1500150500000,BRASIL,40
...,...,...,...,...,...,...
510,511,NCT NATION: TO THE WORLD IN CINEMAS,NCT NATION: TO THE WORLD IN CINEMAS,E2300498800000,CORÉIA DO SUL,52
511,512,JOURNEY TO BETHLEHEM,JORNADA PARA BELEM,E2300507800000,ESTADOS UNIDOS,72
512,513,PRISCILLA,PRISCILLA,E2300508800000,ESTADOS UNIDOS,1
513,514,SEVENTEEN TOUR 'FOLLOW' TO JAPAN: LIVE VIEWING,SEVENTEEN TOUR 'FOLLOW' TO JAPAN: LIVE VIEWING,E2300549400000,CORÉIA DO SUL,28


In [35]:
filme.pais_origem = filme.pais_origem.astype("category")
filme.from_distribuidora = filme.from_distribuidora.astype("category")

In [37]:
print("Linhas e Colunas: ",filme.shape)
print("Primeiras 5 linhas:\n",filme.head())
print("indices das colunas",filme.columns)
print("tipos de dados apos editado: ", filme.info())

print("Moda da coluna: ",filme.pais_origem.value_counts().head(1))
print("Moda da coluna: ",filme.from_distribuidora.value_counts().head(1))

Linhas e Colunas:  (515, 6)
Primeiras 5 linhas:
    id               titulo_original titulo_br         cpb_roe pais_origem  \
0   1                 DURVAL DISCOS      None  B0200001000000      BRASIL   
1   2                     NOSSO LAR      None  B1001259400000      BRASIL   
2   3                       REMOÇÃO      None  B1301918400000      BRASIL   
3   4  HOJE EU QUERO VOLTAR SOZINHO      None  B1402094700000      BRASIL   
4   5        O HOMEM DE LAGOA SANTA      None  B1500150500000      BRASIL   

  from_distribuidora  
0                 24  
1                  3  
2                 51  
3                 24  
4                 40  
indices das colunas Index(['id', 'titulo_original', 'titulo_br', 'cpb_roe', 'pais_origem',
       'from_distribuidora'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0 

# Distribuidora

In [15]:
distribuidora

,id,nome,cnpj
0,1,O2 PRODUÇÕES ARTÍSTICAS E CINEMATOGRÁFICAS LTDA.,67.431.718/0001-03
1,2,WARNER BROS. (SOUTH) INC.,33.015.827/0001-28
2,3,THE WALT DISNEY COMPANY (BRASIL) LTDA.,73.042.962/0001-87
3,4,FREESPIRIT DISTRIBUIDORA DE FILMES LTDA.,07.616.202/0001-01
4,5,A2 DISTRIBUIDORA DE FILMES LTDA EPP,18.338.912/0001-33
...,...,...,...
67,68,F D DA SILVA - ME,24.578.765/0001-71
68,69,LUDWIG MAIA ARTHOUSE DISTRIBUIDORA DE FILMES LTDA,05.249.852/0001-30
69,70,GARDA DISTRIBUIDORA DE FILMES LTDA,34.433.654/0001-20
70,71,SA DISTRIBUIDORA DE CONTEÚDO AUDIOVISUAL LTDA,33.356.343/0001-42


In [39]:
print("Linhas e Colunas: ",distribuidora.shape)
print("Primeiras 5 linhas:\n",distribuidora.head())
print("indices das colunas",distribuidora.columns)
print("tipos de dados apos editado: ", distribuidora.info())

Linhas e Colunas:  (72, 3)
Primeiras 5 linhas:
    id                                              nome                cnpj
0   1  O2 PRODUÇÕES ARTÍSTICAS E CINEMATOGRÁFICAS LTDA.  67.431.718/0001-03
1   2                         WARNER BROS. (SOUTH) INC.  33.015.827/0001-28
2   3            THE WALT DISNEY COMPANY (BRASIL) LTDA.  73.042.962/0001-87
3   4          FREESPIRIT DISTRIBUIDORA DE FILMES LTDA.  07.616.202/0001-01
4   5               A2 DISTRIBUIDORA DE FILMES LTDA EPP  18.338.912/0001-33
indices das colunas Index(['id', 'nome', 'cnpj'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      72 non-null     int64 
 1   nome    72 non-null     object
 2   cnpj    72 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.8+ KB
tipos de dados apos editado:  None
